## Deep Biaffine Attention for Neural Dependency Parsing
这是斯坦福博士生Dozat提出的一种基于图的依存句法分析方法，主要解决两个问题：1、哪两个节点连依存弧；2、弧的标签是什么。具体文章见https://openreview.net/pdf?id=Hk95PK9le 和 https://web.stanford.edu/~tdozat/files/TDozat-CoNLL2017-Paper.pdf 。

### 一、依存句法分析
描述语法有两种主流观点：
 - (1) 短语结构文法。用固定数量的rule分解句子为短语和单词、分解短语为更短的短语或单词。一个取自WSJ语料库的短语结构树示例：
<img src="http://localhost:9999/files/my_learning/pic/deep_biaffine_parser_wsj.jpg" width="400" height="300" />
 - (2) 依存结构。用单词之间的依存关系来表达语法。箭头的尾部是dependent（被修饰的主题），箭头指向的是head(修饰语)。
 <img src="http://localhost:9999/files/my_learning/pic/deep_biaffine_parser_universal_ dependency.jpg" width="500" height="400" />
 
 依存句法分析有几个条件:
 - ROOT只能被一个词依赖。
 - 无环
 
 依存句法分析有两种方法：
 - Transition-base parsing.
 - Graph-base parsing.

### 二、Deep Biaffine Parser结构
#### 1. 训练数据格式
依存句法分析的训练数据一般为CoNLL格式的语料，以.conll结尾。CONLL标注格式包含10列，分别为：<br/>
********
ID   FORM    LEMMA   CPOSTAG POSTAG  FEATS   HEAD    DEPREL  PHEAD   PDEPREL
********
前8列的含义如下：

 序列号|类型|含义
 --|--|--
 1|ID|当前词在句子中的序号
 2|FORM|当前词语或标点
 3|LEMMA|当前词语（或标点）的原型或词干，一般此列与FORM相同
 4|CPOSTAG|当前词语的词性（粗粒度）
 5|POSTAG|当前词语的词性（细粒度）
 6|FEATS|句法特征
 7|HEAD|当前词语的中心词
 8|DEPREL|当前词语与中心词的依存关系

依存句法分析使用的数据包括:第1列、第2列、第4列、第7列和第8列。

#### 2. 模型输入数据
依存句法分析的输入数据包括:
- 词语。可以加载预训练的词向量或字符向量。
- 词性。词性经过词性标注模型得到。

#### 3. 输入层 (MLP)
一个句子的词语和词性经过embedding，输入到(多层)BiLSTM,然后每个词汇的BiLSTM输出结果通过4个不同的ReLU(MLP)层得到4种特殊的向量表示：

- $h^{arc-dep}$ : 该词作为dependent(子节点)寻找head(父节点)。
- $h^{arc-head}$ : 该词作为head寻找其所有的dependents 。
- $h^{rel-dep}$ : 该词作为dependent决定其label(弧标签)。
- $h^{rel-head}$ : 该词作为head决定其所有dependents的label。

输入层的结构如下:
<img src="http://localhost:9999/files/my_learning/pic/deep_biaffine_parser_emb.png" width="600" height="400" />

假设一个句子有n个词汇，给定n个词汇的embedding向量$(v_{1}^{word},...,v_{n}^{word})$和相应的词性$(v_{1}^{tag},...,v_{n}^{tag})$,将两个向量列表成对拼接起来:<br/>
> $x_{i} = v_{i}^{word} \bigoplus v_{i}^{tag}$

将它们输入到以$r_{0}$为初始化向量的BiLSTM结构,获得每个词的输出向量:<br/>
> $r_{i} = BiLSTM(r_{0} , (x_{1} , ... , x_{n}))_{i}$ <br/>
> $ h_{i},c_{i} = split(r_{i})$

然后通过4个独立的MLP获得上述4种向量表示:
> $h_{i}^{(arc-dep)} = MLP^{(arc-dep)}(h_{i})$ <br/>
> $h_{i}^{(arc-head)} = MLP^{(arc-head)}(h_{i})$ <br/>
> $h_{i}^{(rel-dep)} = MLP^{(rel-dep)}(h_{i})$ <br/>
> $h_{i}^{(rel-head)} = MLP^{(rel-head)}(h_{i})$ <br/>

#### 4. 预测弧
对于第 $i$ 个词，其它所有词作为其head的分数为：<br/>
> $s_{i}^{(arc)}=H^{(arc-head)}W^{(arc)}h_{i}^{(arc-dep)}+H^{(arc-head)}b^{T(arc)}$ <br/>
<br/>
> $y_{i}^{'(arc)}=arg \underset{j}{max} s_{ij}^{(arc)}$ <br/>

假设MLP得到的4个向量长度均为 $h$ ,则 $H^{(arc-head)}$ 的维度为 $n\times h$ ,参数 $W^{(arc)}$ 维度为 $h\times h$ ,偏置参数向量 $b^{T(arc)}$ 的维度为 $h\times 1$,因此 $s_{i}^{arc}$ 的维度为 $n\times 1$ ，维度具体计算过程如下:
> $(n\times 1)=(n\times h)*(h\times h)*(h\times 1)+(n\times h)*(h\times 1)$

将偏置合并到 $W$,可以用以下结构表示：
<img src="http://localhost:9999/files/my_learning/pic/deep_biaffine_parser_arc.png" width="600" height="400" />

如果将词向量长度为 $h$ 的句子中 $n$ 个词放在一起，组成 $n\times h$ 的矩阵$H^{(arc-dep)}$，则其他所有词的head的分数为:<br/>
> $S^{(arc)}=H^{(arc-head)}(W^{(arc)}\oplus b^{(arc)})(H^{(arc-dep)} \oplus 1)^{T}$ <br/>

对应的维度是:
> $ (n\times n)=(n\times h)*(h\times (h+1))*((h+1)\times n)$

对于第 $i$ 个词，其它所有词作为其head的分数为矩阵 $S$ 第 $i$ 列的最大值：<br/>
>$y_{i}^{'(arc)}=arg \underset{j}{max} S_{ji}^{(arc)}$ <br/>

#### 5.预测弧标签
在确定了词 $i$ 的 $head$ 之后，使用另一个网络预测这条弧的label:
> $s_{i}^{(rel)}=h_{y_{i}^{'(arc)}}^{T(rel-head)}U^{(rel)}h_{i}^{(rel-dep)}+W^{(rel)}(h_{i}^{(rel-dep)}\oplus h_{y_{i}^{'(arc)}}^{(rel-head)})+b^{(rel)} $ <br/>
><br/>
> $y_{i}^{'(rel)}=arg \underset{j}{max} s_{ij}^{(rel)}$ <br/>

若 $label$ 有 $r$ 种标签，$h_{y_{i}^{'(arc)}}^{T(rel-head)}$ 和 $h_{i}^{(rel-dep)}$ 的维度是 $h \times 1$, $U^{(rel)}$ 的维度是 $r \times h \times h$ , $W^{(rel)}$ 的维度是 $r \times 2h$ , $b$ 维度是 $r \times 1$ ,则维度计算过程如下:
> $(r \times 1) = (1 \times h)(r \times h \times h)(h \times 1)+(r \times 1\times 2h)(2h \times 1)+(r \times 1\times 1)$

将偏置合并到 $U$ ,可以用以下结构表示：
<img src="http://localhost:9999/files/my_learning/pic/deep_biaffine_parser_arc_label.jpg" width="600" height="400" />

计算时将 $U$ 中的 $r$ 个 $(h+1) \times (h+1)$ 矩阵分别与 $h_{y_{i}}^{(rel-arc)} \oplus 1$ 和 $h_{i}^{(rel-dep)} \oplus 1$ 进行矩阵运算。<br/>

若同时计算句子 $n$ 个词汇所对应的弧标签，则可表示为:
> $S^{(rel)}= (H_{y^{'(arc)}}^{(rel-head)} \oplus 1)U^{(rel)}(H^{(rel-dep)} \oplus 1)^{T} $ 

对应的维度是:
>$ (r\times n \times n)=(n \times (h+1))(r\times (h+1) \times (h+1))((h+1)\times n) $

则第 $i$ 个词的 $label$ 得分 ,可以先根据弧的 $head$ , 从 $S^{(rel)}$ 的第2维度(dim=1)找到第 $j$ 行，再从第1维度(dim=0)的 $r$ 个得分中找到最大值:
>$ s_{i}^{'(rel)}=S^{(rel)}[: , y_{i}^{'(arc)},i] $ <br/>
><br/>
> $s_{i}^{(rel)}=arg \underset{j}{max}s_{ij}^{(rel)}$

#### 6. 损失函数和预测
训练时，以上述两个分类器的softmax交叉熵损失作为优化目标。在测试时，通过为每个可能的根迭代地解决环问题生成一棵符合约束的树，然后选择其中总分最高的。比如用Chu-Liu/Edmonds算法。

### 三、Pytorch构建模型
实际模型训练时，每次迭代采用batch_size样本进行模型训练。具体代码见https://github.com/daandouwe/biaffine-dependency-parser。

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class BiAffineParser(nn.Module):

    def __init__(self, word_vocab_size, word_emb_dim,
                 tag_vocab_size, tag_emb_dim, emb_dropout,
                 lstm_hidden, lstm_num_layers, lstm_dropout,
                 mlp_arc_hidden, mlp_lab_hidden, mlp_dropout,
                 num_labels, criterion=nn.CrossEntropyLoss()):
        super(BiAffineParser, self).__init__()

        # Embeddings
        self.word_embedding = nn.Embedding(word_vocab_size, word_emb_dim, padding_idx=0)
        self.tag_embedding = nn.Embedding(tag_vocab_size, tag_emb_dim, padding_idx=0)
        self.emb_dropout = nn.Dropout(p=emb_dropout)

        # LSTM
        lstm_input = word_emb_dim + tag_emb_dim
        self.lstm = nn.LSTM(input_size=lstm_input, hidden_size=lstm_hidden,
                            num_layers=lstm_num_layers, batch_first=True,
                            dropout=lstm_dropout, bidirectional=True)

        # Arc MLPs
        mlp_input = 2*lstm_hidden
        self.arc_mlp_h = MLP(mlp_input, mlp_arc_hidden, 2, 'ReLU', mlp_dropout)
        self.arc_mlp_d = MLP(mlp_input, mlp_arc_hidden, 2, 'ReLU', mlp_dropout)
        # Label MLPs
        self.lab_mlp_h = MLP(mlp_input, mlp_lab_hidden, 2, 'ReLU', mlp_dropout)
        self.lab_mlp_d = MLP(mlp_input, mlp_lab_hidden, 2, 'ReLU', mlp_dropout)

        # BiAffine layers
        self.arc_biaffine = BiAffine(mlp_arc_hidden, 1)
        self.lab_biaffine = BiAffine(mlp_lab_hidden, num_labels)

        # Loss criterion
        self.criterion = criterion

    def forward(self, words, tags):
        """
        Compute the score matrices for the arcs and labels.
        """
        words = self.word_embedding(words)
        tags = self.tag_embedding(tags)
        x = torch.cat((words, tags), dim=-1)
        x = self.emb_dropout(x)

        h, _ = self.lstm(x)

        arc_h = self.arc_mlp_h(h)
        arc_d = self.arc_mlp_d(h)
        lab_h = self.lab_mlp_h(h)
        lab_d = self.lab_mlp_d(h)

        S_arc = self.arc_biaffine(arc_h, arc_d)
        S_lab = self.lab_biaffine(lab_h, lab_d)
        return S_arc, S_lab

    def arc_loss(self, S_arc, heads):
        """
        Compute the loss for the arc predictions.
        """
        S_arc = S_arc.transpose(-1, -2)                     # [batch, sent_len, sent_len]
        S_arc = S_arc.contiguous().view(-1, S_arc.size(-1)) # [batch*sent_len, sent_len]
        heads = heads.view(-1)                              # [batch*sent_len]
        return self.criterion(S_arc, heads)

    def lab_loss(self, S_lab, heads, labels):
        """
        Compute the loss for the label predictions on the gold arcs (heads).
        """
        heads = heads.unsqueeze(1).unsqueeze(2)             # [batch, 1, 1, sent_len]
        heads = heads.expand(-1, S_lab.size(1), -1, -1)     # [batch, n_labels, 1, sent_len]
        S_lab = torch.gather(S_lab, 2, heads).squeeze(2)    # [batch, n_labels, sent_len]
        S_lab = S_lab.transpose(-1, -2)                     # [batch, sent_len, n_labels]
        S_lab = S_lab.contiguous().view(-1, S_lab.size(-1)) # [batch*sent_len, n_labels]
        labels = labels.view(-1)                            # [batch*sent_len]
        return self.criterion(S_lab, labels)

In [1]:
import torch
from torch import nn
from torch.nn import init

class BiAffine(nn.Module):
    """
    Biaffine attention layer.
    """
    def __init__(self, input_dim, output_dim):
        super(BiAffine, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.U = nn.Parameter(torch.FloatTensor(output_dim, input_dim, input_dim))
        init.xavier_uniform(self.U)

    def forward(self, Rh, Rd):
        Rh = Rh.unsqueeze(1)
        Rd = Rd.unsqueeze(1)
        S = Rh.matmul(self.U).matmul(Rd.transpose(-1, -2))
        # S = Rh @ self.U @ Rd.transpose(-1, -2)
        return S.squeeze(1)